In [5]:
from tavily import TavilyClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Literal, Dict, Any
from urllib.parse import urlparse
import os

from langsmith import traceable


tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
llm = ChatOpenAI(model="gpt-4o", temperature=0)


class JobFitAnalysis(BaseModel):
    """The structured output we want from the LLM."""
    match_reasoning: str = Field(description="Internal reasoning: Explain WHY this is a match or mismatch.")
    job_title: str = Field(description="The specific role title.")
    company_name: str = Field(description="Company name. If not found, use domain name.")
    
    matching_skills: List[str] = Field(description="List of skills found in BOTH resume and job.")
    missing_skills: List[str] = Field(description="List of skills found in job but MISSING in resume.")
    
    fit_category: Literal["Perfect Fit", "Good Match", "Irrelevant"] = Field(
        description="Perfect Fit = Role matches & user has most skills. Good Match = Role matches but user needs upskilling."
    )

def get_diverse_results(results):
    domain_map = {}
    unique_results = []
    for item in results:
        domain = urlparse(item['url']).netloc
        if domain not in domain_map: domain_map[domain] = []
        domain_map[domain].append(item)
    
    max_items = max(len(v) for v in domain_map.values())
    domains = list(domain_map.keys())
    for i in range(max_items):
        for domain in domains:
            if i < len(domain_map[domain]): unique_results.append(domain_map[domain][i])
            
    return unique_results[:6]


@traceable(run_type="tool", name="Tavily Job Search") 
def step_1_fetch_raw_jobs(profile) -> List[Dict[str, Any]]:
    loc = profile.location if profile.location and profile.location.lower() != "unknown" else "Remote"
    print(f"\n📍 Search Location: {loc} (Press Enter to confirm or type new location)")
    user_loc = input("   > ")
    if user_loc.strip(): loc = user_loc.strip()

    query = f"{profile.target_role} jobs in {loc} hiring now requirements"
    print(f"\n🔍 [Step 1] Fetching raw jobs from Tavily for: '{query}'...")
    
    response = tavily_client.search(
        query=query, 
        topic="general", 
        max_results=12,
        search_depth="advanced",
        include_raw_content=True
    )
    
    raw_results = get_diverse_results(response.get("results", []))
    print(f"✅ [Step 1] Raw search complete. List optimized for diversity.")
    return raw_results


@traceable(run_type="chain", name="Extract Job Details") 
def step_2_extract_details(raw_jobs: List[Dict[str, Any]], profile) -> List[Dict[str, Any]]:
    print(f"\n🧠 [Step 2] Sending unstructured content to LLM for extraction...")
    structured_llm = llm.with_structured_output(JobFitAnalysis)
    processed_jobs = []
    
    for i, job in enumerate(raw_jobs):
        if len(job.get("raw_content", "")) < 200: continue
            
        print(f"   ... Analyzing Job {i+1}...")
        
        prompt_text = f"""
        Analyze this raw job description text.
        
        --- CANDIDATE ---
        Target Role: {profile.target_role}
        Skills: {', '.join(profile.technical_skills)}
        
        --- RAW UNSTRUCTURED JOB TEXT ---
        Title: {job['title']}
        URL: {job['url']}
        CONTENT: {job['raw_content'][:6000]}
        """
        
        try:
            
            prompt = ChatPromptTemplate.from_messages([
                ("system", "You are a Job Data Parser. Convert raw text into structured lists."),
                ("human", "{text}")
            ])
            analysis = (prompt | structured_llm).invoke({"text": prompt_text})
            
            if analysis.fit_category != "Irrelevant":
                job_data = analysis.model_dump()
                job_data['url'] = job['url']
                processed_jobs.append(job_data)
        except Exception as e:
            continue
            
    return processed_jobs

print("Search completed")

Search completed


In [ ]:

from IPython.display import display, Markdown

try:
    
    raw_job_list = step_1_fetch_raw_jobs(profile)
    
    if raw_job_list:
        
        final_jobs = step_2_extract_details(raw_job_list, profile)
        
        
        display(Markdown("## 🚀 **Final Job Opportunities Report**"))
        
        if not final_jobs:
            print("❌ No matches found after processing.")
        
        for job in final_jobs:
            
            match_str = ", ".join(job['matching_skills']) if job['matching_skills'] else "None"
            miss_str = ", ".join(job['missing_skills']) if job['missing_skills'] else "None"
            
            
            job_card = f"""
### 📌 {job['job_title']} | *{job['company_name']}*
**✅ Matching Skills:** {match_str}  
**⚠️ Missing Skills:** {miss_str}  
🔗 [**Link to Job Application**]({job['url']})
___
"""
            display(Markdown(job_card))
            
    else:
        print("❌ No raw jobs found in Step 1.")

except Exception as e:
    print(f"❌ Execution Error: {e}")


# I ENTERED GURGAON AS MY NEW LOCATION TO TEST OUT THE HUMAN-IN-THE-LOOP FUNCTIONALITY


📍 Search Location: New York, USA (Press Enter to confirm or type new location)

🔍 [Step 1] Fetching raw jobs from Tavily for: 'UX Designer jobs in Gurgaon hiring now requirements'...
✅ [Step 1] Raw search complete. List optimized for diversity.

🧠 [Step 2] Sending unstructured content to LLM for extraction...
   ... Analyzing Job 1...
   ... Analyzing Job 2...
   ... Analyzing Job 3...
   ... Analyzing Job 4...
   ... Analyzing Job 5...
   ... Analyzing Job 6...


## 🚀 **Final Job Opportunities Report**


### 📌 Sr. UX Designer | *ixigo*
**✅ Matching Skills:** Adobe Suite, Sketch, InVision  
**⚠️ Missing Skills:** User Research, User-Centered Design, Design Strategy, End-to-End UX Design, Iterative Design & Testing, Cross-Functional Collaboration, Design System & Standards, Stakeholder Communication, Problem Identification & Resolution  
🔗 [**Link to Job Application**](https://designproject.io/jobs/jobs/sr-ux-designer-at-ixigo-2hs253)
___



### 📌 UI/UX Designer | *Destiny HR Group Services*
**✅ Matching Skills:** Sketch, Adobe Suite  
**⚠️ Missing Skills:** Figma, Adobe XD, Adobe Illustrator, Photoshop, InDesign  
🔗 [**Link to Job Application**](https://www.destinyhrgroup.com/job/jobs-for-ui-ux-designer-in-gurgaon)
___



### 📌 UX Designer | *Jobsora*
**✅ Matching Skills:** Adobe Suite, Sketch, InVision  
**⚠️ Missing Skills:** Figma, Photoshop, Illustrator, After Effects  
🔗 [**Link to Job Application**](https://in.jobsora.com/jobs-ux-designer-gurugram-gurgaon-district-haryana-state)
___



### 📌 UI/UX Designer | *Systellar Technologies*
**✅ Matching Skills:** HTML5, CSS, Adobe Suite  
**⚠️ Missing Skills:** Figma, Adobe XD, Photoshop, Illustrator, User-centered design principles  
🔗 [**Link to Job Application**](https://www.iitjobs.com/job/uiux-designer-gurgaon-haryana-india-systellar-technologies-104667)
___



### 📌 UI/UX Designer | *naukri.com*
**✅ Matching Skills:** HTML5, CSS, JavaScript, Adobe Suite, Sketch, InVision, Balsamiq  
**⚠️ Missing Skills:** None  
🔗 [**Link to Job Application**](https://www.naukri.com/ui-ux-designer-jobs-in-gurgaon)
___



### 📌 UX / UI Designer - Digital Health | *Benovymed Healthcare*
**✅ Matching Skills:** CSS  
**⚠️ Missing Skills:** AI  
🔗 [**Link to Job Application**](https://www.glassdoor.ie/Job/gurgaon-haryana-ux-designer-new-grad-jobs-SRCH_IL.0,15_IC2921225_KO16,36.htm)
___
